<a href="https://colab.research.google.com/github/Pame1509/EstadisticaActs/blob/main/Copy_of_Actividad2_Estadistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocesar Archivo para obtener supereditor y filtrar los datos

In [9]:
install.packages("haven")
library(haven)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [29]:
# Acceder al archivo es_wiki.txt cargado como publico desde Google Drive
system("gdown --id 1HbHI7LPL0dJM-h7GdAQB8FZHUiZsFi0r")

In [1]:
# Input and output file paths
input_file <- "es_wiki.txt"
output_file <- "cleaned_es_wiki.txt"

# Open input and output files
con_in <- file(input_file, "r")
con_out <- file(output_file, "w")

# Function to clean lines with mismatched double quotes
clean_line <- function(line) {
  # Check if the line has an odd number of double quotes
  if (sum(charToRaw(line) == 0x22) %% 2 != 0) {
    # Fix the mismatched quotes by adding a closing quote
    line <- paste0(line, '"')
  }
  return(line)
}

# Process the file line by line
while (TRUE) {
  line <- readLines(con_in, n = 1)
  if (length(line) == 0) {
    break  # End of file
  }
  cleaned_line <- clean_line(line)
  writeLines(cleaned_line, con_out)
}

# Close input and output files
close(con_in)
close(con_out)

# Read the cleaned file using read.csv
data <- read.csv(output_file, sep = ",", header = FALSE)



In [10]:
data <- read.csv("cleaned_es_wiki.txt", header = FALSE, sep=" ")

In [14]:
head(data)

,V1,V2,V3
,<chr>,<chr>,<chr>
1,Gregg_Sulkin,87.217.240.27,1287843598
2,Gregg_Sulkin,87.217.240.27,1287843504
3,Sonangol,Sentoan,1287843154
4,Matt_&_Kim,Shining.Star,1287842935
5,Matt_&_Kim,Shining.Star,1287842910
6,Ricardo_Aparicio_y_Aparicio,Petronas,1287842871


In [13]:
data <- data[, -(4:12)]

In [16]:
# Renombrar las columnas
colnames(data) <- c("Web", "Editor", "Tiempo")
head(data)

,Web,Editor,Tiempo
,<chr>,<chr>,<chr>
1,Gregg_Sulkin,87.217.240.27,1287843598
2,Gregg_Sulkin,87.217.240.27,1287843504
3,Sonangol,Sentoan,1287843154
4,Matt_&_Kim,Shining.Star,1287842935
5,Matt_&_Kim,Shining.Star,1287842910
6,Ricardo_Aparicio_y_Aparicio,Petronas,1287842871


In [17]:
# Contar las frecuencias de cada valor en la columna "Editor"
editor_freq <- table(data$Editor)

# Ordenar las frecuencias en orden descendente
order_freq <- sort(editor_freq, decreasing = TRUE)

# Mostrar los editores con más registros
head(order_freq, n = 10)





                    'religion']        'salud']    Diegusjaimes       Matdrodes 
         683245          132892          118659          113791          105879 
    'politica',     'religion',        Petronas CommonsDelinker            Dodo 
          93900           54703           53586           53385           51350 

In [4]:
# Filtrar por el super editor Deisgusjaimes
filter <- data[data$Editor == "Diegusjaimes", ]

# Guardar el archivo filtradi
write.csv(filter, file = "Diegusjaimes.csv", row.names = FALSE)

head(filter)


ERROR: ignored

# Analisis series Temporales

In [10]:
install.packages("Forecast")
install.packages("ImputeTS")
install.packages("vars")
install.packages("timsac")
install.packages("data.table")
install.packages("zoo")
install.packages("tseries")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘Forecast’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Warning message:
“Perhaps you meant ‘forecast’ ?”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘ImputeTS’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Warning message:
“Perhaps you meant ‘imputeTS’ ?”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘zoo’, ‘strucchange’, ‘urca’, ‘lmtest’, ‘sandwich’


Installing package into ‘/usr/local/lib/R/site-library’

In [13]:
# Librerias
#library(Forecast)
#library(ImputeTS)
library(vars)
library(timsac)
library(data.table)
library(zoo)
library(tseries)

Loading required package: MASS

Loading required package: strucchange

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: sandwich

Loading required package: urca

Loading required package: lmtest

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



In [19]:
df <- read.csv("/content/Diegusjaimes.csv", header = TRUE)
head(df)

,Web,Editor,Tiempo
,<chr>,<chr>,<int>
1,Cadena_productiva,Diegusjaimes,1267486279
2,Cadena_productiva,Diegusjaimes,1254941786
3,Cadena_productiva,Diegusjaimes,1249133434
4,Cadena_productiva,Diegusjaimes,1249133332
5,Jaime_Vivanco,Diegusjaimes,1287498621
6,Art_Attack,Diegusjaimes,1260487271


In [20]:
# Obtener el rango de tiempo de todas las ediciones
min_time <- min(df$Tiempo)
max_time <- max(df$Tiempo)

# Crear una secuencia de tiempo en segundos
Tiempo <- seq(min_time, max_time, by = 1)

# Crear una lista de ceros
Edicion <- integer(length(Tiempo))

# Marcar las ediciones del super-editor como 1 en la serie temporal
edicion_segundos <- as.integer(df$Tiempo)
index <- edicion_segundos - min_time + 1
Edicion[index] <- 1

# Crear un DataFrame con la serie temporal
serie_temporal <- data.frame(Tiempo = Tiempo, Edicion = Edicion)

# Resultado
head(serie_temporal)

,Tiempo,Edicion
,<dbl>,<dbl>
1,1210624192,1
2,1210624193,0
3,1210624194,0
4,1210624195,0
5,1210624196,0
6,1210624197,0


In [29]:
# Contar cuántas veces aparece el valor 1 en la columna "Edicion"
contador_edicion_1 <- sum(serie_temporal$Edicion == "1")

# Mostrar el resultado
print(contador_edicion_1)


[1] 113555


In [30]:
# Crear la serie temporal
serie_temporal <- ts(serie_temporal$Edicion, start = min(serie_temporal$Tiempo), frequency = 1)


Time Series:
Start = 1210624192 
End = 1287793301 
Frequency = 1 
    [1] 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   [37] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   [73] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [109] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [145] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [181] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [217] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [253] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [289] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [325] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [361] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [397] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 